In [65]:
import sys
sys.path.append('/global/homes/s/sleak/.local/cori/3.6-anaconda-4.4/lib/python3.6/site-packages')
import rdflib

#sys.path.append('/global/homes/s/sleak/Monitoring/Resilience/LogSet/prototypes')
#import Graph    
#
## force re-read of Graph.py in case I've edited it:
#import importlib
#importlib.reload(Graph)
#
#url = 'nersc-catalog.ttl'
#vocab = '../etc/vocab.ttl'
##Graph.read(url, vocab=vocab)
#Graph.construct(url, local_only=False)
#
#Graph.dump()
def dump(graph):
    print(graph.serialize(format='n3').decode('ascii'))

import urllib.request
def parse_line_by_line(url, fmt):
    """ rdflib is really uninformative about the location of the error
        when it can't parse a file, so use this to parse line-by-line 
        and look for errors
        (hmm, does not work, fails on valid files)
    """
    f = urllib.request.urlopen(url)
    g = rdflib.ConjunctiveGraph()
    linecount=0
    for line in f.readlines():
        linecount += 1
        print("parsing line {0:d}: {1}".format(linecount,line))
        rdftext = line.strip()
        if not rdftext: continue
        try:
            g.parse(data=rdftext, format=fmt)
        except:
            print("failed to parse line {0:d} of {1}".format(linecount,url))
            print(line)
            print(g.serialize())
            raise
    print("finished parsing {0:s} successfully!".format(url))
    
import os
def construct(*catalog_urls, spider=False):
    graph = rdflib.ConjunctiveGraph()
    unparsed = set(catalog_urls)
    print(unparsed)
    parsed = set()
    q_peers = 'SELECT ?uri WHERE { ?cat logset:peers ?uri . }'
    while len(unparsed) > 0:
        for url in unparsed:
            print("current dir is" + os.getcwd())
            print(url)
            if os.path.isfile(url):
                print("yes, found " + str(url))
            else:
                print("no, did not find " + str(url))
            fmt = rdflib.util.guess_format(url)
            
            parse_line_by_line(url, format)
            graph.parse(url, format=fmt)
            
            
                
            parsed.add(url)
        unparsed = set()   
        if spider:
            for peer in graph.query(q_peers):
                url = peer['uri']
                if not url in parsed:
                    unparsed.add(url)
                    
    q_logsets = ''' SELECT ?uri 
                    WHERE {
                        ?cat dcat:dataset ?uri . 
                    }'''
    for logset in graph.query(q_logsets):
        print("found a logset url: {0}".format(str(logset)))
        url = logset['uri']
        fmt = rdflib.util.guess_format(url)
        graph.parse(url, format=fmt)
    return graph

# quick sanity check:
#parse_line_by_line('file:/global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc-entities.ttl', 'turtle')
g1 = rdflib.ConjunctiveGraph().parse("file:/global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index.ttl", format='turtle')
print("got this far...")
print(g1.serialize(format='n3').decode('ascii'))

graph = construct('nersc-entities.ttl', spider=True)
dump(graph)
                
    

[ns for ns in graph.namespaces()]

BadSyntax: at line 14 of <>:
Bad syntax (Prefix "logset:" not bound) at ^ in:
"...b'elf and set a prefix:\n@prefix : <examples/index#> .\n:\n    a '^b'logset:LogSet ;\n    dct:description "a partial sample of cor'..."

In [59]:
# find other catalogs
def spider(graph):
    q_cats = '''SELECT ?uri
                 WHERE {
                    ?cat a dcat:Catalog .
             '''
    q_peers = '''SELECT ?uri
                 WHERE {
                    ?cat a dcat:Catalog .
                    ?cat logset:peers ?uri .
                 } '''
    for peer in graph.query(q_peers):
        print(peer['uri'])
        graph.parse(peer['uri'], format=rdflib.util.guess_format(peer['uri']))
        
#while 
spider(graph)
dump(graph)
spider(graph)
dump(graph)


NameError: name 'graph' is not defined

In [ ]:
# find datasets, read them, and if they are LogSets, add them to the graph: